# Exploring the mechanism of `Exchange`

In [ ]:
from datetime import datetime, timedelta

import os
os.chdir("..")  # path to the base directory of RL4MM
from RL4MM.orderbook.create_order import create_order, OrderDict
from RL4MM.orderbook.models import Order, Orderbook, LimitOrder, MarketOrder, Cancellation, Deletion
from RL4MM.orderbook.Exchange import *
from RL4MM.orderbook.helpers import *

# Jedi not working
%config Completer.use_jedi = False

In [ ]:
ticker="FAKE"
start = datetime(2019,1,2,10,35,45)
delta = timedelta(microseconds = 100000)

## To manage an Orderbook, create an ```Exchange```

each exchange automatically initialises an ```Orderbook``` to keep track of.

In [ ]:
exchange = Exchange(ticker)
orderbook = exchange.central_orderbook

## Make some tools to create orders of different types

In [ ]:
def get_limit_order(timestamp:datetime, price:float, volume:int, direction:str, internal_id: int):
    return LimitOrder(timestamp = timestamp, price = price, volume=volume, direction=direction, ticker = "MSFT,=", internal_id=internal_id, external_id=None, is_external= True)
def get_market_order(timestamp:datetime, volume:int, direction:str, internal_id: int=None):
    return MarketOrder(timestamp = timestamp, volume=volume, direction=direction, ticker = "MSFT,=", internal_id=internal_id, external_id=None, is_external= True)
def get_cancellation(timestamp:datetime, price: float, volume:int, direction:str, internal_id:int):
    return Cancellation(timestamp = timestamp, price = price, volume=volume, direction=direction, ticker = "MSFT,=",  internal_id = internal_id, external_id=None, is_external= True)
def get_deletion(timestamp:datetime, price: float, direction:str, internal_id):
    return Deletion(timestamp = timestamp, price = price, volume = None, direction=direction, ticker = "MSFT", internal_id = internal_id, external_id=None, is_external= True)

## Submitting limit orders

In [ ]:
limit_buy_price_vol_tuples = ((99.8,10), (99.85, 50), (99.87, 100), (99.9, 80), (99.95, 100), (99.98, 200))
limit_sell_price_vol_tuples = ((100.1,20), (100.05, 30), (100.04, 60), (100.02, 80), (100.01, 150))
internal_id = 1
ts = start
limit_orders = list()
for price, vol in limit_buy_price_vol_tuples:
    limit_orders.append(get_limit_order(ts, price, vol, "buy", internal_id))
    internal_id +=1
    ts +=delta
for price, vol in limit_sell_price_vol_tuples:
    limit_orders.append(get_limit_order(ts, price, vol, "sell", internal_id))
    internal_id +=1
    ts +=delta       

In [ ]:
for order in limit_orders:
    exchange.process_order(order)
visualise_orderbook(exchange.central_orderbook)

## Market orders

In [ ]:
market_order = get_market_order(direction="sell", volume=150, timestamp=ts)
ts += delta
exchange.process_order(market_order)
visualise_orderbook(exchange.central_orderbook)

### Trying to place a market order that takes all the volume from the book, and more

Note that perhaps this is not the desired behaviour and we should let the market order take all the liquidity (as we currently do) but then just ignore the rest. To discuss.

In [ ]:
market_order = get_market_order(direction="sell", volume=700, timestamp=ts)
ts += delta

try:
    exchange.process_order(market_order)
except EmptyOrderbookError:
    print("Volume too large!")    

## Cancellations and Deletions

In [ ]:
# reset orderbook
exchange.central_orderbook = exchange.get_empty_orderbook()
for order in limit_orders:
    exchange.process_order(order)

In [ ]:
exchange.central_orderbook.buy

In [ ]:
# We cancel half of the buy order at level 99.9
order = exchange.central_orderbook.buy[99.9][0]
cancellation = get_cancellation(timestamp=ts, price = order.price, direction="buy", internal_id=order.internal_id, volume=order.volume/2)
ts += delta

In [ ]:
exchange.process_order(cancellation)

In [ ]:
visualise_orderbook(exchange.central_orderbook)

In [ ]:
# Delete best order
order = exchange.central_orderbook.buy[99.98][0]
deletion = get_deletion(timestamp=ts, price = order.price, direction="buy", internal_id=order.internal_id)
ts += delta

In [ ]:
deletion

In [ ]:
exchange.process_order(deletion)

In [ ]:
visualise_orderbook(exchange.central_orderbook)

## Crossing the spread with a limit order

In [ ]:
# reset orderbook
exchange.central_orderbook = exchange.get_empty_orderbook()
for order in limit_orders:
    exchange.process_order(order)
visualise_orderbook(exchange.central_orderbook)

In [ ]:
limit_cross = get_limit_order(start, 100.03, 300, "buy", internal_id)
internal_id +=1
exchange.process_order(limit_cross)
visualise_orderbook(exchange.central_orderbook)